In [ ]:
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph

In [19]:
n = 100000
tau1 = 3
tau2 = 1.5
mu = 0.1
G = LFR_benchmark_graph(
    n, tau1, tau2, mu, average_degree=16, min_community=20, seed=10
)

#nodes, tau1, tau2, mu, average_degree

In [20]:
configuration = {
    'n':[100, 1000, 10000],
    'tau1': [3, 3.5],
    'tau2': [1.5, 2],
    'mu': [0.1, 0.2]
}

In [21]:

configurations_possible = []

def get_possible_configs(conf, index, current):

    if index == len(conf):
        configurations_possible.append(current)
        return 

    here = list(conf.keys())[index]

    for value in conf[here]:
        current[here] = value
        get_possible_configs(conf, index + 1, current)


get_possible_configs(configuration, 0, {})



In [22]:
for configuration in configurations_possible:
    print(configuration)
    G = LFR_benchmark_graph(configuration['n'], configuration['tau1'], \
        configuration['tau2'], configuration['mu'], average_degree=16, min_community=20, seed=10)
    #todo: acrescentar aqui as metricas
    break

{'n': 10000, 'tau1': 3.5, 'tau2': 2, 'mu': 0.2}


In [23]:
import math
print(G.number_of_edges())
print(math.sqrt(2*G.number_of_edges()))

90428
425.27167787192224


In [24]:
communities = {frozenset(G.nodes[v]["community"]) for v in G}


In [25]:
print(len(communities))

164


In [26]:
#for x in communities:
#    print(x)

In [27]:
from collections import defaultdict

size_comm = defaultdict(int)
for x in communities:
    size_comm[len(x)]+=1

size_comm = {k: v for k, v in sorted(size_comm.items(), key=lambda item: item[0], reverse=True)}
print(size_comm)

{401: 1, 394: 1, 244: 1, 241: 1, 236: 1, 219: 1, 215: 1, 202: 1, 183: 1, 174: 1, 157: 1, 154: 1, 153: 1, 146: 1, 145: 1, 136: 1, 133: 1, 119: 1, 105: 1, 103: 1, 97: 2, 95: 1, 93: 1, 91: 2, 90: 1, 89: 4, 88: 1, 87: 1, 82: 1, 81: 1, 79: 2, 76: 1, 73: 2, 71: 1, 69: 1, 68: 2, 67: 1, 65: 1, 60: 2, 59: 2, 58: 1, 57: 1, 56: 2, 55: 2, 54: 3, 53: 3, 52: 1, 50: 2, 49: 2, 48: 3, 47: 1, 46: 1, 44: 2, 43: 4, 42: 1, 41: 1, 40: 3, 39: 2, 37: 2, 36: 4, 35: 2, 34: 5, 33: 3, 32: 2, 31: 3, 30: 4, 29: 3, 28: 3, 27: 2, 26: 2, 25: 6, 24: 3, 23: 9, 22: 6, 21: 8, 20: 12}


In [28]:
import community.community_louvain as community_louvain

In [29]:
partion = community_louvain.best_partition(G)

found_comm = defaultdict(int)
for node, community in partion.items():
    found_comm[community]+=1

# (community_nr, nr_nodes)

another = defaultdict(int)

for cnr, c in found_comm.items():
    another[c]+=1

another = {k: v for k, v in sorted(another.items(), key=lambda item: item[0], reverse=True)}


print(another)

{401: 1, 394: 1, 244: 1, 241: 1, 236: 1, 219: 1, 215: 1, 202: 1, 183: 1, 174: 1, 157: 1, 154: 1, 153: 1, 146: 1, 145: 1, 143: 1, 141: 1, 139: 1, 136: 1, 133: 1, 132: 1, 131: 1, 129: 1, 125: 1, 124: 1, 122: 1, 120: 1, 119: 2, 118: 2, 116: 1, 112: 1, 111: 1, 109: 1, 108: 1, 107: 1, 104: 1, 103: 1, 102: 2, 100: 1, 98: 1, 97: 1, 96: 1, 95: 1, 94: 1, 93: 2, 92: 1, 91: 1, 90: 1, 89: 1, 88: 2, 86: 1, 83: 1, 82: 2, 79: 2, 78: 1, 77: 1, 76: 2, 74: 2, 73: 2, 72: 1, 71: 1, 70: 1, 69: 1, 62: 1, 60: 2, 59: 2, 58: 1, 56: 3, 55: 2, 50: 1, 48: 1, 44: 1, 42: 1, 40: 2, 35: 1}


In [30]:
temp_dict = defaultdict()

c = 1
for y in communities:
    temp_dict[c] = y
    c+= 1



In [31]:
coisas = defaultdict(set)
for node, community in partion.items():
    coisas[community].add(node) # (community_nr, nodes)



valor_bonito = []
assigned_ones = defaultdict(list)# sets from the louvain that are assigned as most similar to the same set of the lrf
valores = []
inesperado = []

for k ,y in temp_dict.items(): # (com_nr, set community) do lrf
    higher = -1
    target_set = set()
    contador_inesperado = 0
    for blah, x in coisas.items(): # (comm_nr2, set community) do louvain

        temp = x & y
        if higher < len(temp):
            higher = len(temp)
            target_set = temp
            rrr = blah
            contador_inesperado+=1
    
    if higher !=-1:
        valor_bonito.append(target_set)
        assigned_ones[rrr].append(k)
        valores.append(k)

    if contador_inesperado > 1:
        inesperado.append(k)


c = 0
for x in valor_bonito:
    c+=len(x)


assigned_ones = {k: v for k, v in sorted(assigned_ones.items(), key=lambda item: len(item[1]), reverse=True)}

print(c/G.number_of_nodes())
print(len(inesperado), inesperado)
print(assigned_ones)


0.9999
162 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164]
{73: [48, 85, 91, 108], 77: [2, 25, 154], 29: [3, 111, 161], 67: [10, 11, 97], 38: [14, 15, 81], 20: [16, 152, 155], 82: [17, 40, 76], 62: [19, 131, 158], 54: [20, 54, 68], 49: [26, 77, 119], 47: [31, 118, 159], 48: [38, 104, 136], 36: [41, 123, 160], 78: [42, 141, 146], 16: [44, 52, 

In [32]:
contador = 0
for a, b in assigned_ones.items():
    contador += len(b)

print(len(assigned_ones))
print(len(coisas))


91
91


In [34]:

intersection_value_metric = 0

for a, b in assigned_ones.items():
    z = set()
    for v in b:
        z = z.union(temp_dict[v])
    
    #assert len(z) == len(coisas[a])
    tamanho = z.intersection(set(coisas[a]))
    #assert len(z) == len(tamanho)
    intersection_value_metric += len(tamanho)

print(intersection_value_metric/G.number_of_nodes())

0.9999


In [36]:
import networkx.algorithms.community as nxcomm
nxcomm.modularity(G,coisas.values())


0.6898126315711915

In [37]:
nxcomm.modularity(G,temp_dict.values())


0.6887294909621239

In [ ]:
max(partion.values())

275